In [49]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [61]:
import numpy as np
from base.MachiKoro.env import *
#0 Array bias trong mỗi trận
#1 Array bias sau 10000k trận 
#2 Lưu số trận 
#3 lưu hệ số đã dùng 
#4 Tỷ lệ thắng khi đấu 10k trận 
#5 Trigger để train 10000k trận 
#6 Đếm số lần yếu liên tiếp
#7 Lưu array cho policy yếu 
#8 State của trận đấu
#9 Action của trận đấu
#10 Reward của trận đấu
#11 Số trận lưu state
#12 Trigger policy yếu
#13 Lưu array của decision tree
#14 Lưu giá trị để chia Node (TBC active win -TBC active lose)
#15 Lưu thứ tự node của decision tree
perx = [np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,getActionSize())),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,2)),np.zeros((1,2)),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(4)]),np.zeros((200000,getStateSize())),np.zeros((200000,1)),np.zeros((200000,1)),np.zeros((1,1)),np.zeros((1,3)),np.array([np.random.rand(getStateSize())-0.55 for _ in range(4)]),np.zeros((200000,4)),np.zeros((4,1)),np.zeros((1,getActionSize())),np.zeros((1,getActionSize())),np.zeros((1,getActionSize())),np.zeros((1,getActionSize()))]
                         #0                                                      #1                   #2              #3                                                            #4              #5               #6                #7                                                               #8                                   #9               #10                   #11           #12              #13                                                             #14              #15                 #16                                   #17                  #18                          #19

# Hàm xây cây 


In [62]:
import numpy as np
@njit
def Build_decision_tree(per):
    # replace -1 with 1 and 0
    for i in range(len(per[10])):
        if per[10][i] == -1:
            for j in range(i,len(per[10])):
                if per[10][j] != -1:
                    per[10][i] = per[10][j]
                    break
    # replace -1 with 0
    for i in range(len(per[10])):
        if per[10][i] == -1:
            per[10][i] = 0
            
    # Calculate the active match and non-active match
    for j in range(4):
        for i in range(200000):
            per[14][i][j] += (per[8][i]@per[13][j])

    #Calculate the active win and acitve lose
    for j in range(4):
        for i in range(200000):
            if per[14][i][j] > 1 and per[10][i] == 1:
                per[15][j] += 1
            if per[14][i][j] > 1 and per[10][i] == 0:
                per[15][j] -= 1 
    return 0

@njit 
def ChooseActionDecisionTree(state,per):
    # Find build the decision tree
    # Node 1 np.min(score)
    # Node 2 second min score
    # Node 3 third min score
    # Node 4 max scored
    l = np.argmax(per[15])
    k = np.argmin(per[15])
    m1 =0
    m2 = 0
    for i in range(4):
        if i != l and i != k and m1 == 0:
            m1 += i
        if i != l and i != k and m1 != 0:
            m2 += i
    k1 = 0
    k2 = 0
    
    if  per[15][m1]>per[15][m2]:
        k1 = m1
        k2 = m2
    else:
        k1 = m2
        k2 = m1

    #Node1
    if state@per[13][l] > 1:
        return l
    else:
        #Node2
        if state@per[13][k1] > 1:
            return k1
        else:
            #Node3
            if state@per[13][k2] > 1:
                return k2
            #Node4
            else:
                if state@per[13][k] > 1:
                    return k
                else:
                    return -1


In [144]:
perx[14]

array([[ -3.83556315,   1.2213083 ,  -5.07323939, -15.03526716],
       [ -5.55638527,  -3.19920865,  -4.81989855, -13.80676021],
       [ -3.39807471,  -1.20889308,  -5.02391345, -15.67584002],
       ...,
       [ -8.37683896,  -2.68534107,  -1.27436763, -14.25855748],
       [ -9.13397441,  -6.89908887,  -0.60686886, -13.77990031],
       [ -8.94235838,  -7.08275101,  -0.40433249, -11.75219874]])

In [46]:
list(perx[15])

[array([2924.21796689]),
 array([1148.21796689]),
 array([488.21796689]),
 array([1411.21796689])]

In [58]:
[perx[15][1]/np.sum(perx[15])][0]

array([0.19227103])

In [69]:
[perx[15][1]/np.max(perx[15]),perx[15][0]/np.sum(perx[15]),perx[15][2]/np.sum(perx[15]),perx[15][3]/np.sum(perx[15])]

[array([0.39265813]),
 array([0.48966522]),
 array([0.08175292]),
 array([0.23631082])]

In [87]:
np.random.choice(np.array([1,0,2,3]),p=[0.25,0.25,0.25,0.25])

3

# Agent Hoàn Chỉnh

In [70]:
import numpy as np
@njit()
def p_bias(state,per):
    # Chuyển Mode gồm 3 mode mode 0 - train bias , mode 1 - test bias , mode 2 - train policy yếu ,mode 3 - test policy yếu,mode 4 - lưu state
    if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      # Lưu lại kết quả thắng
      per[5][0][1] = 0 
      per[5][0][1] += (per[4][0][0])
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Reset số lần yếu liên tiếp
      per[6][0][0] = 0 
    elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[6][0][0] +=1 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 

    # Bắt đầu mode lưu state
    if per[6][0][0] == 3 and per[11][0][0] < 200000 :
      # Chuyển sang lưu state 
      per[5][0][0] = 4
      per[2][0][0] = 0
      # Rest số trận thắng và số trận đấu 
      per[4][0][1] = 0 
      per[4][0][0] = 0
      
    # Hết mode lưu state chuyển sang train policy cho các arr
    if per[5][0][0] == 4 and per[11][0][0] == 200000:
      per[5][0][0] = 2

    # Bất đầu mode train và test policy arr
    if per[11][0][0] == 200000:
      if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        #
        if np.sum(per[16][0]) !=0:
          per[7][0] = np.copy(per[16][0]/np.max(per[16][0]))        
        if np.sum(per[17][0]) !=0:
          per[7][1] = np.copy(per[17][0]/np.max(per[17][0]))
        if np.sum(per[18][0]) !=0:
          per[7][2] = np.copy(per[18][0]/np.max(per[18][0]))
        if np.sum(per[19][0]) !=0:
          per[7][3] = np.copy(per[19][0]/np.max(per[19][0])) 
        #
        # Lưu lại kết quả thắng
        per[5][0][1] = 0 
        per[5][0][1] += (per[4][0][0])
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
        # Chuyển sang train
        per[12][0][2] = np.random.choice(np.array([1,0,2,3]))
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))

      elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
        per[12][0][2] = np.random.choice(np.array([1,0,2,3]))


    # reset per[1][0] sau 100k trận
    if int(per[2][0][0]) == 100000 and per[11][0][0] < 200000:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 1
    # reset per[1][0] sau 10k trận
    if int(per[2][0][0]) == 100000 and per[11][0][0] == 200000:
      #
      if np.sum(per[16][0]) !=0:
        per[7][0] = np.copy(per[16][0]/np.max(per[16][0]))        
      if np.sum(per[17][0]) !=0:
        per[7][1] = np.copy(per[17][0]/np.max(per[17][0]))
      if np.sum(per[18][0]) !=0:
        per[7][2] = np.copy(per[18][0]/np.max(per[18][0]))
      if np.sum(per[19][0]) !=0:
        per[7][3] = np.copy(per[19][0]/np.max(per[19][0]))
      #
      per[2][0][0] = 0
      per[5][0][0] = 3
      per[12][0][2] = np.random.choice(np.array([1,0,2,3]))

    if per[11][0][0] == 200000 and per[12][0][0] == 0:
      per[12][0][0] +=1 
      per[12][0][0] += Build_decision_tree(per)

    # Chế độ train bias
    if int(per[2][0][0]) < 100000 and per[5][0][0] == 0  :
      actions = getValidActions(state)
      actions *= per[0][0]
      action = np.argmax(actions)
      if getReward(state) == 1 :
          per[1] += per[0][0]
      if getReward(state) == 0:
          per[0][0] = np.random.rand(getActionSize())

    # Chế độ test bias
    elif int(per[2][0][0]) <10000  and per[5][0][0] == 1 : 
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0])    
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  

    # Chế độ train policy arr
    elif int(per[2][0][0]) < 100000 and per[5][0][0] == 2 and per[11][0][0] == 200000 :
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      Check_yeu = ChooseActionDecisionTree(state,per)
      if Check_yeu == per[12][0][2]:
        per[12][0][1] = -1
        actions *= per[0][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
        
      if getReward(state) == 1 and per[12][0][1] != 0:
        per[int(per[12][0][2]+16)] += per[0][0]
        per[12][0][1] = 0
        
      if getReward(state) == 0 and per[12][0][1] != 0:
        per[0][0] = np.random.rand(getActionSize())
        per[12][0][1] = 0

    # chế độ test policy yếu
    elif int(per[2][0][0]) <10000 and  per[5][0][0] == 3 and per[11][0][0] == 200000:
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      node = ChooseActionDecisionTree(state,per)
      
      if node != -1:
        actions *= per[7][node]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)

      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  
    

    # Báo chuyển sang chế độ lưu state và action
    if per[5][0][0] == 4 and per[11][0][0] < 200000:
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0]) 


    # Lưu state và action
    if getReward(state) != -2 and per[11][0][0] < 200000 and per[5][0][0] == 4:
        for i in range(getStateSize()):
          per[8][int(per[11][0][0])][i] += state[i]        
        per[9][int(per[11][0][0])][0] += int(action)
        per[10][int(per[11][0][0])][0] += int(getReward(state))
        per[11][0][0] += 1

    # đếm số ván đấu 
    if getReward(state) != -1 :
        per[2][0][0] += 1
    return action,per

win, bias = numba_main_2(p_bias, 1000000,perx,0)

In [67]:
perx[15]

array([[3.]])

In [68]:
perx[7]

array([[0.94014545, 0.4532549 , 0.7302798 , 0.08740159, 0.33859935,
        0.57176434, 0.70240171, 0.45240649, 0.73415086, 0.2315212 ,
        0.85301818, 0.55944826, 0.43727301, 0.76844644, 0.11484158,
        0.02631527, 0.66596741, 0.00151795, 0.50001383, 0.87419997,
        0.48629188, 0.23302959, 0.01097628, 0.68135116, 0.9045445 ,
        0.27340282, 0.58144376, 0.53678347, 0.90525584, 0.22543674,
        0.72893048, 0.65060838, 0.43913916, 0.37161236, 0.15179053,
        0.69537189, 0.52754845, 0.77687775, 0.14313024, 0.8061265 ,
        0.41946068, 0.98926282, 0.36575031, 0.47818657, 0.46825995,
        0.74191437, 0.77584181, 0.83767177, 0.19857772, 0.52802868,
        0.49856027, 0.69303932, 0.20162423, 0.48928911],
       [0.18406709, 0.2349893 , 0.74727661, 0.76974028, 0.28728223,
        0.95500627, 0.78416414, 0.04855777, 0.14829326, 0.93421291,
        0.06507147, 0.66399121, 0.09893419, 0.11787808, 0.37442309,
        0.43516948, 0.08665236, 0.44677825, 0.40561766, 0.9

# Test lại tỷ lệ thắng của level 1 game MachiKoro

**Khi chưa khắc phục policy yếu :24551** 


**Khi đã khắc phục policy yếu :27961**

In [ ]:
@njit()
def test(state,per):
    actions = getValidActions(state)
    # Check xem policy có yếu không 
    node = ChooseActionDecisionTree(state,per)
    if node != -1:
      actions *= per[7][node]
      action = np.argmax(actions)
    else:
      actions *= per[3][0]
      action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(test, 10000,perx,1)
print(win)

2474


In [209]:
perx[15]

array([[ 270.],
       [  -7.],
       [-255.],
       [ -74.]])

# Tạo cây rỗng ==> Tạo các tính chất trước 

In [7]:
%cd /Users/trancatkhanh/Downloads/GAME
import numpy as np
from base.MachiKoro.env import *
per_state= [np.zeros((200000,getStateSize())),np.zeros((200000,1)),np.zeros((1,1))]

@njit()
def tao_arr(state,per):
    actions = getValidActions(state)
    # Check xem policy có yếu không 
    l = np.where(actions ==1)[0]
    action = np.random.choice(l)
    if per[2][0][0] < 200000:
        for i in range(getStateSize()):
            per[0][int(per[2][0][0])][i] += state[i]
        per[1][int(per[2][0][0])][0] += getReward(state)
        per[2][0][0]+=1 
    return action,per
@njit
win, bias = numba_main_2(tao_arr, 100000,per_state,0)
print(win)

/Users/trancatkhanh/Downloads/GAME
271


In [6]:
for i in range(len(per_state[1])):
    if per_state[1][i] == -1:
        for j in range(i,len(per_state[1])):
            if per_state[1][j] != -1:
                per_state[1][i] = per_state[1][j]
                break
# replace -1 with 0
for i in range(len(per_state[1])):
    if per_state[1][i] == -1:
        per_state[1][i] = 0
np.where(per_state[1]==1)[0]

array([  131,   132,   133, ..., 49775, 49776, 49777])

In [11]:
l@per_state[0][1]

54.41581469623884

In [72]:
l = np.array([np.random.rand(getStateSize())-0.55 for _ in range(4)])
k = np.zeros((200000,4))
# save all the value of l @ each array of per_state[0]
for j in range(4):
    for i in range(len(per_state[0])):
        k[i][j] += (per_state[0][i]@l[j])
score = np.zeros((4,1))
# if k[i][j] > 1 and per_state[1][i] == 1 then score[j] += 1 
# if k[i][j] > 1 and per_state[1][i] == 0 then score[j] -= 1
for j in range(4):
    for i in range(len(per_state[0])):
        if k[i][j] > 1 and per_state[1][i] == 1:
            score[j] += 1
        if k[i][j] > 1 and per_state[1][i] == 0:
            score[j] -= 1 


In [74]:
score

array([[ -42.],
       [ -11.],
       [ -57.],
       [-329.]])